<a href="https://colab.research.google.com/github/sb2356-iiitr/ML_Projects/blob/main/RecursiveFeatureElimination/rfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement Recursive Feature Elimination

Predict product purchase for the Bank Telemarketing dataset

In [ ]:
# Import libraries
import pandas as pd

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving bank.csv to bank.csv
User uploaded file "bank.csv" with length 4928746 bytes


In [ ]:
# Read file
f = pd.read_csv('bank.csv')

In [ ]:
f.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
# Drop duration column
f = f.drop("duration", axis = 1)

In [ ]:
# Split columns into Dependent and Independent variables
x = f.iloc[:, :-1]
y = f.iloc[:, -1]

In [ ]:
# Create dummy variables
x = pd.get_dummies(x, drop_first=True)
y = pd.get_dummies(y, drop_first=True)

In [ ]:
# Split the dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = \
train_test_split(x, y, test_size = 0.3, random_state = 1234, stratify = y)

In [ ]:
# Random forest classifier
from sklearn.ensemble import RandomForestClassifier

rfc1 = RandomForestClassifier(random_state=1234)
rfc1.fit(X_train, Y_train)
Y_predict1 = rfc1.predict(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [ ]:
# Score and evaluate the model
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(Y_test, Y_predict1)
score1 = rfc1.score(X_test, Y_test)
print(cm1)
print(score1)

[[10647   318]
 [  988   404]]
0.8943109168892126


In [ ]:
# Apply Recursive Feature Elimination and select 30 features
from sklearn.feature_selection import RFE

In [ ]:
# Get an estimator object
rfc2 = RandomForestClassifier(random_state=1234)
rfe = RFE(estimator=rfc2, n_features_to_select=30, step=1)
rfe.fit(x, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RFE(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                     class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto',
                                     max_leaf_nodes=None, max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=None,
                                     oob_score=False, random_state=1234,
                                     verbose=0, warm_start=False),
    n_features_to_select=30, step=1, verbose=0)

In [ ]:
x_train_rfe = rfe.transform(X_train)
x_test_rfe = rfe.transform(X_test)

In [ ]:
# Fit the random forest classifier to the new train and test with 80 features
rfc2.fit(x_train_rfe, Y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1234,
                       verbose=0, warm_start=False)

In [ ]:
# Test the model with new Test dataset
Y_predict = rfc2.predict(x_test_rfe)

In [ ]:
# Score and evaluate the new model
from sklearn.metrics import confusion_matrix
cm_rfe = confusion_matrix(Y_test, Y_predict)
score_rfe = rfc2.score(x_test_rfe, Y_test)
print(cm_rfe)
print(score_rfe)

[[10638   327]
 [  979   413]]
0.8943109168892126


In [ ]:
# Get column names
columns = list(x.columns)

In [ ]:
# Get the ranking of the features.
# Ranking 1 for selected features
ranking = rfe.ranking_

In [ ]:
# Get the feature important scores
feature_importance = rfc1.feature_importances_

In [ ]:
# Create the dataframe of the Features selected, Ranking and Feature Importance
rfe_selected = pd.DataFrame()

rfe_selected = pd.concat([pd.DataFrame(columns),
                          pd.DataFrame(ranking),
                          pd.DataFrame(feature_importance)],
                         axis=1)
rfe_selected.columns = ["Feature Name", "Ranking", "Feature Importance"]

In [ ]:
rfe_selected

,Feature Name,Ranking,Feature Importance
0,age,1,1.708099e-01
1,campaign,1,8.396704e-02
2,pdays,1,3.095854e-02
3,previous,1,1.910895e-02
4,emp.var.rate,1,2.206491e-02
5,cons.price.idx,1,2.318935e-02
6,cons.conf.idx,1,2.788468e-02
7,euribor3m,1,1.210910e-01
8,nr.employed,1,6.266152e-02
9,job_blue-collar,1,1.401534e-02
